In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!pip install -U flask-cors
!pip install flask-ngrok
!ngrok authtoken ""

In [ ]:
from flask import Flask,request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin

from io import BytesIO
import torch
from PIL import Image
import base64
from google.colab import files

In [ ]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
run_with_ngrok(app)  

@app.route('/')
def hello_world():
  return '<h1>Hello, Abhishek!</h1>'

@app.route("/img",methods=['POST', 'GET'])
@cross_origin()
def img():
  org=bytes_in = request.get_json()["img"].replace(" ","+")
  

  device = "cuda" if torch.cuda.is_available() else "cpu"
  model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
  face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)
  image_format="png"
  bytes_in = base64.b64encode(bytes(bytes_in, 'utf-8'))
  bytes_in1=base64.decodebytes(bytes_in)
  with open('decoded_image.png', 'wb') as file_to_save:
      decoded_image_data = base64.decodebytes(bytes_in1)
      file_to_save.write(decoded_image_data)


  file = open("decoded_image.png","rb")
  bytes_in1 = file.read()
  file.close()

  im_in = Image.open(BytesIO(bytes_in1)).convert("RGB")
  im_out = face2paint(model, im_in, side_by_side=False)
  buffer_out = BytesIO()
  im_out.save(buffer_out, format=image_format)
  bytes_out = buffer_out.getvalue()
  bs64_out = base64.b64encode(bytes_out)
  return str(bs64_out).replace(" ","+")[2:-1]

if __name__ == '__main__':
  app.run()